In [2]:
import numpy as np
from scipy.optimize import minimize

In [15]:
def physics_loss_numpy(params, x, alpha=0.143e-6, beta=1/1000, nu=0.000001):
    nx, ny = x.shape[0], x.shape[1]
    print(x.shape)
    dx = 10 / nx
    dy = 1 / ny

    # Assuming params is a flat array, we'll need to reshape it to get u, v, p, T
    u = params[:nx*ny].reshape(nx, ny)
    v = params[nx*ny:2*nx*ny].reshape(nx, ny)
    p = params[2*nx*ny:3*nx*ny].reshape(nx, ny)
    T = params[3*nx*ny:].reshape(nx, ny)
    # Central difference for interior points for u_x
    u_x_internal = (u[ 2:, 1:-1] - u[ :-2, 1:-1]) / (2 * dx)
    u_y_internal= (u[ 1:-1,2:] - u[ 1:-1,:-2]) / (2 * dy)
    u_xx_internal= (u[ 2:, 1:-1] - 2 * u[ 1:-1, 1:-1] + u[ :-2, 1:-1]) / (dx ** 2)
    u_yy_internal=(u[ 1:-1,2:] - 2 * u[ 1:-1, 1:-1] + u[ 1:-1,:-2]) / (dy ** 2)
        
    v_x_internal = (v[ 2:, 1:-1] - v[ :-2, 1:-1]) / (2 * dx)
    v_y_internal= (v[ 1:-1,2:] - v[ 1:-1,:-2]) / (2 * dy)
    v_xx_internal= (v[ 2:, 1:-1] - 2 * v[ 1:-1, 1:-1] + v[ :-2, 1:-1]) / (dx ** 2)
    v_yy_internal=(v[ 1:-1,2:] - 2 * v[ 1:-1, 1:-1] + v[ 1:-1,:-2]) / (dy ** 2)
        
    T_x_internal = (T[ 2:, 1:-1] - T[ :-2, 1:-1]) / (2 * dx)
    T_y_internal= (T[ 1:-1,2:] - T[ 1:-1,:-2]) / (2 * dy)
    T_xx_internal= (T[ 2:, 1:-1] - 2 * T[ 1:-1, 1:-1] + T[ :-2, 1:-1]) / (dx ** 2)
    T_yy_internal=(T[ 1:-1,2:] - 2 * T[ 1:-1, 1:-1] + T[ 1:-1,:-2]) / (dy ** 2)

    p_x_internal=(p[ 2:, 1:-1] - p[ :-2, 1:-1]) / (2 * dx)
    p_y_internal=(p[ 1:-1,2:] - p[ 1:-1,:-2]) / (2 * dy)
        # Continuity Loss
    u_internal=u[ 1:-1, 1:-1]
        

    v_internal=v[ 1:-1, 1:-1]


        
        

        

    L_cont1 = u_x_internal + v_y_internal

    # Create a zeros array with the shape of u_x_internal
    f = np.zeros_like(u_x_internal)

    # Compute Mean Squared Error (MSE) in numpy
    L_cont = np.mean((L_cont1 - f)**2)

    # Compute L_mom_u1, L_mom_v1, and L_heat1
    L_mom_u1 = u_internal*u_x_internal + v_internal*u_y_internal + beta*p_x_internal - nu*(u_xx_internal + u_yy_internal)
    L_mom_u = np.mean((L_mom_u1 - f)**2)

    L_mom_v1 = u_internal*v_x_internal + v_internal*v_y_internal + beta*p_y_internal - nu*(v_xx_internal + v_yy_internal)
    L_mom_v = np.mean((L_mom_v1 - f)**2)

    L_heat1 = u_internal*T_x_internal + v_internal*T_y_internal - alpha*(T_xx_internal + T_yy_internal)
    L_heat = np.mean((L_heat1 - f)**2)

    u_avg = 10**-3
    u_inlet_expected = u_avg*1.5*(1 - 4*(y[ 0, :]**2))
    T_wall = 350
    T_center = 293
    T_inlet_expected = T_wall + (T_center - T_wall) * (1 - 4*(y[ 0, :]**2))

    # Loss due to boundary conditions
    left_T_bc1 = T[ 0, :]
    left_T_bc = np.mean((left_T_bc1 - T_inlet_expected)**2)

    top_T_bc1 = T[ :, 0]
    T_wallvec = np.full_like(top_T_bc1, T_wall)
    top_T_bc = np.mean((top_T_bc1 - T_wallvec)**2)

    bottom_T_bc1 = T[ :, -1]
    bottom_T_bc = np.mean((bottom_T_bc1 - T_wallvec)**2)

    boundaryzero = np.zeros_like(u[:, :, 0])

    left_u_bc = np.mean((u[ 0, :] - u_inlet_expected)**2)
    top_u_bc = np.mean((u[ :, 0] - boundaryzero)**2)
    bottom_u_bc = np.mean((u[ :, -1] - boundaryzero)**2)

    vzeroleft = np.zeros_like(v[ 0, :])
    left_v_bc = np.mean((v[ 0, :] - vzeroleft)**2)
    top_v_bc = np.mean((v[ :, 0] - boundaryzero)**2)
    bottom_v_bc = np.mean((v[ :, -1] - boundaryzero)**2)

    right_p_bc = np.mean((p[ -1, :] - vzeroleft)**2)

    bc_loss = np.linalg.norm([left_T_bc, top_T_bc, bottom_T_bc, 2*left_u_bc, top_u_bc, bottom_u_bc, 2*left_v_bc, top_v_bc, bottom_v_bc, right_p_bc])

    # You'll end with computing the total loss:
    L = np.linalg.norm(L_mom_u + L_mom_v + L_cont + L_heat +bc_loss)

    return L



In [16]:
num_points = 129  # number of grid points
# Generate evenly spaced points in the Y dimension
y_values = np.linspace(-0.5, 0.5, num_points)
x_values=np.linspace(0,10,num_points)
# Repeat these points in the X dimension
X_data1=np.tile(x_values,(num_points,1))
Y_data1 = np.tile(y_values, (num_points, 1))
print(Y_data1.shape,X_data1.shape)
print(X_data1[0,:])

(129, 129) (129, 129)
[ 0.        0.078125  0.15625   0.234375  0.3125    0.390625  0.46875
  0.546875  0.625     0.703125  0.78125   0.859375  0.9375    1.015625
  1.09375   1.171875  1.25      1.328125  1.40625   1.484375  1.5625
  1.640625  1.71875   1.796875  1.875     1.953125  2.03125   2.109375
  2.1875    2.265625  2.34375   2.421875  2.5       2.578125  2.65625
  2.734375  2.8125    2.890625  2.96875   3.046875  3.125     3.203125
  3.28125   3.359375  3.4375    3.515625  3.59375   3.671875  3.75
  3.828125  3.90625   3.984375  4.0625    4.140625  4.21875   4.296875
  4.375     4.453125  4.53125   4.609375  4.6875    4.765625  4.84375
  4.921875  5.        5.078125  5.15625   5.234375  5.3125    5.390625
  5.46875   5.546875  5.625     5.703125  5.78125   5.859375  5.9375
  6.015625  6.09375   6.171875  6.25      6.328125  6.40625   6.484375
  6.5625    6.640625  6.71875   6.796875  6.875     6.953125  7.03125
  7.109375  7.1875    7.265625  7.34375   7.421875  7.5       7.578

In [17]:
# Initial parameters (flattened version of u, v, p, T)
grid=np.concatenate((X_data1,Y_data1),-1)
u_initial=np.zeros_like(X_data1)
v_initial=np.zeros_like(X_data1)
p_initial=np.zeros_like(X_data1)
T_initial=np.zeros_like(X_data1)
params_initial = np.concatenate([u_initial.flatten(), v_initial.flatten(), p_initial.flatten(), T_initial.flatten()])

# Optimize
result = minimize(physics_loss_numpy, params_initial, args=(grid,), method='L-BFGS-B')

# Extract the optimized parameters
u_optimized = result.x[:nx*ny].reshape(nx, ny)
v_optimized = result.x[nx*ny:2*nx*ny].reshape(nx, ny)
p_optimized = result.x[2*nx*ny:3*nx*ny].reshape(nx, ny)
T_optimized = result.x[3*nx*ny:].reshape(nx, ny)

(129, 258)


ValueError: cannot reshape array of size 0 into shape (129,258)